In [1]:
import os
import re

In [10]:
directory = r"/Users/humuyao/Downloads/Book3"
folders = ['Y']
# ['7', 'X', 'Y', 'Z']
 
# regex for introductory words - not perfect but works for most cases
introductory_words = ["^(^|[^a-z0-9]*)introduction($|[^a-z0-9]*)$", "(^|[^a-z0-9]*)preface($|[^a-z0-9]*)$", "(^|[^a-z0-9]*)prologue($|[^a-z0-9]*)$", "(chapter|part|^)[^a-z0-9]*1([^a-z0-9]*|$)$", "(chapter|part|^)[^a-z0-9]*one([^a-z0-9]*|$)$", "(chapter|part|^)([^a-z0-9]*|^)i([^a-z0-9]*|$)$", "^([^a-z0-9]*)(1\.( )?)[a-z]*([^0-9]*|$)$"]
# ending_words = ["epilogue", "acknowledgement", "reference", "index"]

def exists_introductory_words(para):
#     print("para: {}".format(para))
    for word in introductory_words:
        if re.search(r'{}'.format(word), para):
#             print("word that matched is {}".format(re.search(r'{}'.format(word), para).group()))
            helper = re.search(r'{}'.format(word), para).group()
            return re.sub(r'[^a-z0-9 ]', '', helper).strip()
    return ""

# def exists_ending_words(para):
# #     print("para: {}".format(para))
#     for word in ending_words:
#         if word in para:
# #             print("word that matched is {}".format(word))
#             return word
#         if "about" in para and "author" in para:
#             return "about author"
#     return ""

for folder in folders:
    subdir = directory + '/' + folder
    for filename in os.listdir(subdir):
        print("filename: {}".format(filename))
        
        if filename.endswith(".txt"):
            filepath = subdir + '/' + filename
            line_num_first = 0
            begin_words = {}
        
            # looking at first 200 non-white lines of file for those with introductory words
            with open(filepath,'r') as f:
                for line in f:
                    if not line.isspace():
                        line_num_first += 1
                    
                    if line_num_first > 200:
                        break
                    
                    word = exists_introductory_words(line.lower())
                    
                    # storing line numbers of lines containing an introductory word
                    if word != "":
                        print("introductory word is: " + word)
                        print("line containing intro word is: " + line.strip())
                
                        if word not in begin_words:
                            begin_words[word] = [line_num_first]
                        else:
                            begin_words[word].append(line_num_first)
            
            read_from = 0
            print("begin_words dict: {}".format(begin_words))
            
            ################ FINDING THE LINE TO START READING FROM ##################
            # if only one introductory word has appeared, start reading from that line
            if len(begin_words) == 1:
                for word in begin_words:
                    read_from = max(begin_words[word])
            
            # more than one introductory word has appeared
            elif len(begin_words) > 1:
                # all values (of key-value pairs) of the dictionary are single elem lists
                # i.e. multiple introductory words have appeared, but they all appear only once
                all_single_element = True 
                
                # all values (of key-value pairs) of the dictionary are multiple elem lists
                # i.e. multiple introductory words have appeared, but they all appear multiple times (usually twice)
                all_multi_elements = True 
                
                for word in begin_words:
                    if len(begin_words[word]) == 1:
                        all_multi_elements = False
                    if len(begin_words[word]) > 1:
                        all_single_element = False
                
                # this means there are keys with single elem lists AND ALSO keys with multiple elem lists
                # i.e. some introductory words appear once, some appeared more than once
                if not all_single_element and not all_multi_elements:
                    for word in begin_words:
                        # only considering lists with 2 elements
                        if len(begin_words[word]) == 2:
                            # idea: start reading from the latest appearance of the earlier introductory word
                            # e.g. 
                            
                            # Contents Page
                            # -------------
                            # INTRODUCTION
                            # Chapter 1
                            # _____________
                            
                            # - actual book
                            # INTRODUCTION (start reading from here)
                            
                            if read_from == 0:
                                read_from = max(begin_words[word])
                            else:
                                read_from = min(max(begin_words[word]), read_from)
                     
                #  all keys in dictionary have one elem lists
                elif all_single_element:
                    # idea: read the latest appearing introductory word
                    # e.g.
                
                    # Preface
                    # Introduction
                    # Chapter 1 (start reading from here)
                    
                    helper = []
                    for word in begin_words:
                        helper += begin_words[word]
                    read_from = max(helper)
                    
                # all keys in dictionary have multiple elem lists
                elif all_multi_elements:
                    # idea: start reading from the latest appearance of the earlier introductory word
                    # e.g. 
                    
                    # Contents Page
                    # -------------
                    # INTRODUCTION
                    # Chapter 1
                    # _____________

                    # - actual book
                    # INTRODUCTION (start reading from here)
                    # Chapter 1
                    
                    helper = []
                    for word in begin_words:
                        helper.append(max(begin_words[word]))
                    
                    read_from = min(helper)
            
            
            print("read_from line is {}".format(read_from))
            print()
            
            line_num = 0
            
            # actually reading the lines - currently only reading first 10 lines of each file
            with open(filepath,'r') as f:
                count = 0
                for line in f:
                    if not line.isspace():
                        line_num += 1
                        
                    if line_num > read_from and not line.isspace() and len(line) > 50 and "." in line:
                        if count > 10: 
                            break
                        print("line is: {}".format(line.strip()))
                        count += 1
        
        print()

filename: You May Now Kill the Bride - R.L. Stine.epub.txt
introductory word is: one
line containing intro word is: One
introductory word is: part one
line containing intro word is: ## Part One
introductory word is: one
line containing intro word is: ## One
begin_words dict: {'one': [9, 63], 'part one': [61]}
read_from line is 63

line is: Ruth-Ann Fear remembered the feel of hands wrapping around her neck from behind. She remembered the startling pressure and how warm the hands were—warm and damp.
line is: She remembered the leap her heart made, the gasp that escaped her throat. The fingers tightening until she struggled for breath.
line is: With a hard twist of her body, she spun around—and stared into Peter Goodman's eyes. She made another sound, this time a cry of recognition.
line is: He lowered his hands, the fingers sliding gently now, tracing a damp path along her throat. His touch now tender. Still not breathing, Ruth-Ann watched a smile form on his lips.
line is: "Did I start

begin_words dict: {'introduction': [7, 76]}
read_from line is 76

line is: The year 2005 was a science-fictional year in several ways. Think, for instance of SF-scale catastrophes and post-catastrophe civilizations. We were just beginning to understand the scale of the tsunami as the year 2005 dawned. The sheer enormity was hard to grasp, much of it first comprehensible through satellite images of the Aceh coastline wiped clean and the churning vortices of the retreating wave on a few videos. The destruction of New Orleans was disaster come to us, not over there in places we might never have heard of, a disaster made at least as much by our own institutions as by nature, and of science-fictional scale. And this time, especially with technology introduced in 2005, we had front row seats. Between the tsunami and Hurricane Katrina, with the launch of Google Earth, Virtual Earth, and similar web sites, it became possible to have a different relationship to the destruction and the press pho

begin_words dict: {'chapter 1': [11]}
read_from line is 11

line is: "And I need you to help find me one. As soon as possible," Heath continued, ignoring his best friend's reaction.
line is: "I must be drunker than I thought," the duke said, dabbing his chin. "For I could have sworn the Earl of Boden just asked me to find him a wife. And if that wasn't strange enough, the aforementioned earl is supposed to be in Liverpool turning everything he touches into gold. He couldn't possibly be in a London ballroom, certainly not without letting me know he was back in town first." Worth sounded affronted, though he was grinning as he said the last.
line is: "Don't be an ass." Heath felt his own mouth twitch with laughter, but managed to tamp down any feelings of levity before they could interfere with his sense of purpose.
line is: "Well, I've missed you too." The duke thumped Heath on the back and signaled to a footman to bring another drink. "Did you just get back?"
line is: "Cold. Miserable.

line is: Dartmouth was playing Princeton in a rather dull affair early that autumn of 1941. Our minds weren't much on the game. Barbara and I had driven East on our honeymoon and were seeing old college mates for the first time since graduation in 1940, when France was falling, Italy was stabbing, and Britain was holding its breath. And not many of us were conscious of the war, because it was going to take the Battle of Britain to get us interested.
line is: "I don't know. It looks as though I won't be called for a year or more."
line is: "Bolte and those guys have left for England to join the Army there. I'm thinking of going, too."
line is: "Well, it seems there is some sort of old American Regiment over there, and the State Department is letting twelve fellows from here join. Sort of an Eagle Squadron set-up, only you go in the Army."
line is: We were on a honeymoon, and I didn't want to talk about such things just then, but I told Braden I would write him about it, because it sound

line is: My high school chemistry teacher was also a forensics investigator. He specialized in arson, burned bodies, and flammable chemicals, and he entertained us with sometimes-gruesome stories from more than twenty-five years of duty. There was the skeleton of a woman, average height—which is to say five-foot-four—somewhere around thirty years old, reduced to blackened bones and cinders in a house fire. He gave us two clues: "For example, the middle finger on her right hand has a large calcification on the top section, like you might have if you wrote heavily with a pencil, for example." He said "for example" at the beginning and end of everything. "For example, she also has a tiny indentation, a notch, in her front tooth, also the right one, for example." It was our job as eager students, wound up by the grisly details, to figure out her occupation.
line is: Work changes you. It shows itself on your body. In the same way that a carpenter's hands are tuned to the nuances of hammer a

...

begin_words dict: {}
read_from line is 0

line is: Henry's second novel, written, like his first, under a pen name, had done well. It had won prizes and was translated into dozens of languages. Henry was invited to book launches and literary festivals around the world; countless schools and book clubs adopted the book; he regularly saw people reading it on planes and trains; Hollywood was set to turn it into a movie; and so on and so forth.
line is: Henry continued to live what was essentially a normal, anonymous life. Writers seldom become public figures. It's their books that rightly hog all the publicity. Readers will easily recognize the cover of a book they've read, but in a café that man over there, is that... is that... well, it's hard to tell—doesn't he have long hair?—oh, he's gone.
line is: When he was recognized, Henry didn't mind. In his experience, the encounter with a reader was a pleasure. After all, they'd read his book and it had an impact, otherwise why would they com

begin_words dict: {}
read_from line is 0

line is: All rights reserved. Except as permitted under the U.S. Copyright Act of 1976, no part of this publication may be reproduced, distributed, or transmitted in any form or by any means, or stored in a database or retrieval system, without the prior written permission of the publisher.
line is: The Poppy name and logo are trademarks of Hachette Book Group, Inc.
line is: The characters, events, and locations in this book are fictitious. Any similarity to real persons, living or dead, is coincidental and not intended by the author.
line is: she's already a triplet, but b may have found a twin. . . .
line is: r would do anything for love... but he won't do that
line is: _It is better to be feared than loved, if you cannot be both._
line is: News flash: This year's juniors are the people to watch. Seniors and college applications? Who cares? Who wants to waste time on boring will-they-or-won't-they-get-in speculation when there's so much party

begin_words dict: {'preface': [34, 83], 'introduction': [37, 102], '1 northern italy': [53], '1 coins of galba and otho': [60], '1': [179]}
read_from line is 83

line is: The present work seeks to provide a plain narrative of the events of one crowded year. Interwoven by cause and effect, framed in time and space, they embraced the whole Mediterranean world and created the second dynasty of imperial Rome. The web is intricate and colourful. That an attempt to retell the story nineteen hundred years later is possible at all is largely due to the chance survival of the early books of Tacitus' _Histories_ , supported by other information in as generous (or as meagre) a bulk as the historian of ancient Rome can now hope to enjoy. But the Long Year provides us with a thousand problems—dark corners into which only a dim light penetrates from feeble candles, and which we think we can explore by inference, conjecture and imagination. We dispose of no exhaustive official records or revealing me

begin_words dict: {'1 yoga i title': [40], 'preface': [79, 100], 'chapter i': [118]}
read_from line is 100

line is: No WORD can bring to Western minds a greater number of esoteric associations than the name India. The enticement of romance, the lure of gold and diamonds, wanderlust, a desire to face the tiger in the jungles of Bengal, the reputed unerring precision of the mongoose in subduing the cobra, or even a sneaking suspicion that there is a catch somewhere in that tall "rope-trick story"; these and many others are mingled together as elements in a hazy picture of India. While there is a basic residuum of truth in such a conception, a correct understanding of the complexity of modern India is impossible without some historical knowledge of the cultural evolution of the country.
line is: The cultural history of India begins with the Aryan invasion. Leaving aside for the moment the vague origins of what is now one of the oldest cultures of the world, we may profitably summarize th

begin_words dict: {'1': [9]}
read_from line is 9

line is: Iris stood at the counter in the school office, detention slip in hand, anticipating a hand-smack from Mr. Lucas, the vice principal. She'd already seen him twice since her enrollment at Climping Academy the previous fall. The first time, she'd been turned in for cutting PE. The second time, she'd been reported for smoking outside study hall. She'd been advised there was a smoking area set aside specifically for students, which she argued was on the far side of campus and impossible to get to between classes. That fell on deaf ears. This was now early January and she'd been reported for violating the school's dress code.
line is: She was willing to admit that detention slips were a poor means of establishing her place in a new school. The younger students wore uniforms, but in the upper grades, clothing was at the discretion of the individual student as long as the overall look was considered within bounds. The way Iris read it

reasons:
1. Section marked as "Introduction" is not content 
2. Structure is unclear (usually no headings / unclear headings)

exceptions:
- Xavier - Richard Heslop (reason 1)
- XPD - Len Deighton (reason 2 - very ambiguous where book begins)
- xo Orpheus (IDK if this is correct)
- XAIPE (reason 2 - there are no proper sentences in this book?) 
- Xenophon_s Cyrus the Great (reason 1)
- X-Men and Philosophy (reason 1)
- 7 Deadly Women (reason 1/2? - starts with author's note; there is no transition heading to the book actually starting)

looked at 80 Y books, the ones which were not perfectly processed are:
1. Year_s Best SF (ALL OF THEM) (reason 1 - and no clear transition into beginning of book; reads a bit extra)
2. Yoga Hotel (reason 2 - there is no clear introductory section; reads everything)
3. You All Grow Up and Leave Me - Piper Weiss (reason 1; reads a bit extra)
4. Yours for Eternity (reason 2 - has no introductory section; right now starts reading from middle of the book where there is a 1. _____ )
5. You_re the One That I Want (reason 2 - no clear introductory section; reads everything)
6. You Must Set Forth at Dawn (reason 2 - first section has a strange heading; might have missed a chunk in front)
7. Yesterday_Once_More (reason 1 - probably read a bit more)
8. You Belong to Me - Johanna Lindsey (reason 2 - there are no headings; reads everything)
9. You Know When the Men Are Gone (reason 2 - there are no headings; reads everything)
10. Yoga - Barbara Miller (reason 1; reads a bit more)
11. You_re on an Airplane - Parker Posey (reason 1; reads a bit more)
12. Young Knowledge - Hyde, Robin (there are too many lines before the introduction?? LOL WTF)
13. You Can_t Keep a Good Woman Down (reason 2 - no headings; reads everything)
14. Young Widower - John W. Evans (reason 2 - no clear sections; started reading from the middle of the book because of 1. _____)
15. Yours 2 Keep - Kay Hooper (hmm this one seems to be a bit strange bcos of parts; reads everything)
16. You Better Not Cry (reason 2 - no headings; reads everything)
17. Yevgeny Onegin (a lot of non-sentences) 
18. Yes, Chef-Samuelsson (format is "PART 1 **** info behind **** " which my regex does not consider; reads everything) 
19. You Deserve Nothing (reason 2 - no headings; reads everything)
20. You Bring the Distant Near (reason 2 - no headings; reads everything)
21. You Have to Stop This - Pseudonymous Bosch (reason 2 - no headings; reads everything)
22. Yellow Sofa, The (format is of "1     -more text here-" which my regex does not consider; reads everything)
23. You Shall Know Our Velocity (reason 2 - no headings; reads everything)
24. You Gotta Get Bigger Dreams - Alan Cumming (reason 2 - no headings; reads everything)

In [6]:
re.search(r'(chapter|part|^)([^a-z0-9]*|^)one(\s)?[^a-z0-9\s]+([\s]?[a-z0-9]*)*([^a-z]*|$)$', "one works damn")